In [ ]:
### CELL ###
# # Unsupervised Feature Learning and Classification with a Restricted Boltzmann Machine (RBM)
#
# This notebook demonstrates how to build and train a Restricted Boltzmann Machine (RBM) from scratch in Python. It uses an RBM to perform **unsupervised feature learning** on the Iris dataset, and then uses the learned features to build a **supervised classification neural network** using logistic regression.
#
# The project is structured to explain the core concepts of RBMs, including the underlying mathematics, the training algorithm (Contrastive Divergence), and how a trained RBM can be integrated into a supervised classification pipeline.
#
# ## Preamble: From Neural Networks to Boltzmann Machines
#
# ### What are Neural Networks?
#
# A neural network is a computational model inspired by the human brain. It is composed of interconnected nodes, or "neurons," organized in layers. The most basic network consists of an **input layer**, one or more **hidden layers**, and an **output layer**. Each connection between neurons has a numerical weight, and each neuron has a bias.
#
# Neural networks learn by adjusting these weights and biases. Information flows through the network in a process called **forward propagation**, where the input data is multiplied by the weights, summed, and passed through an **activation function** at each neuron. The network's output is then compared to the desired output, and the error is calculated. This error is then propagated backward through the network (**backpropagation**) to update the weights and biases in a way that reduces the error.
#
# ### Training vs. Inference
#
# The process described above is known as **training**. During training, the model learns the patterns and relationships within a dataset. After the model has been trained, it can be used for **inference** or **prediction**. This is the process of using the learned model to make predictions on new, unseen data. In this notebook, our RBM is first trained to learn features from the Iris data in an unsupervised manner (training phase), and then its learned features are used to make predictions with a classifier (inference phase).
#
# ### What is a Boltzmann Machine?
#
# A **Boltzmann Machine** is a special type of neural network that differs from traditional feed-forward networks. It's a **generative, stochastic** model.
#
# * **Generative** means it can learn to model the underlying distribution of the data, allowing it to generate new data samples similar to the training data.
#
# * **Stochastic** means its neurons make decisions randomly, based on probabilities.
#
# Boltzmann Machines consist of two types of units: **visible units** ($\mathbf{v}$) and **hidden units** ($\mathbf{h}$). The visible units represent the observable data, and the hidden units learn to discover abstract features or patterns in that data. The behavior of the network is defined by an **energy function**, which describes the "goodness" of a particular configuration of visible and hidden units.
#
# The energy function for a general Boltzmann Machine is given by:
#
# $$E(\mathbf{v}, \mathbf{h}) = -\sum_{i<j} w_{ij} s_i s_j - \sum_i \theta_i s_i$$
#
# Here, $w_{ij}$ are the weights between units $i$ and $j$, and $\theta_i$ is the bias for unit $i$. The state of each unit $s_i$ is a binary value (0 or 1).
#
# A **Restricted Boltzmann Machine (RBM)** is a simpler version where there are **no connections between units in the same layer** (visible-visible or hidden-hidden). This "restriction" simplifies the model and makes its training significantly more efficient.

### CELL ###
# ## 1. Setup and Data Preprocessing
#
# First, we need to load the necessary libraries and the dataset. The Iris dataset is a classic for classification, and we'll use `scikit-learn` to load it and `numpy` for our mathematical operations.

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# --- Load and prepare the data ---

# Load the Iris dataset from a CSV file
# Assuming Iris.csv is in the same directory as the notebook
df = pd.read_csv('Iris.csv')

# Drop the 'Id' column as it's not needed for the model
df = df.drop('Id', axis=1)

# Separate features (X) and target labels (y)
X = df.iloc[:, 0:4].values
y = df.iloc[:, 4].values

# --- Data preprocessing ---

# Scale the features to a common range.
# This is a critical step as RBMs, like many neural networks,
# perform better with normalized input data.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode the target labels.
# This converts categorical labels (e.g., 'Iris-setosa') into a
# binary vector format (e.g., [1, 0, 0]).
onehot_encoder = OneHotEncoder(sparse_output=False)
y_encoded = onehot_encoder.fit_transform(y.reshape(-1, 1))

# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42
)

### CELL ###
# ## 2. The Restricted Boltzmann Machine (RBM)
#
# An RBM is a type of generative stochastic artificial neural network that can learn a probability distribution over its set of inputs. It is composed of two layers: a **visible layer** (for the input data) and a **hidden layer** (for learning features). The "restricted" part means there are no connections between units within the same layer. This architecture simplifies the training process.
#
# The behavior of an RBM is defined by an **energy function**, which measures how "bad" a given configuration of visible and hidden units is.
#
# ### The Energy Function
#
# For a given configuration of visible units $\mathbf{v}$ and hidden units $\mathbf{h}$, the energy $E(\mathbf{v}, \mathbf{h})$ is defined as:
#
# $$E(\mathbf{v}, \mathbf{h}) = -\sum_{i=1}^{n} b_i v_i - \sum_{j=1}^{m} c_j h_j - \sum_{i=1}^{n} \sum_{j=1}^{m} w_{ij} v_i h_j$$
#
# Where:
# * $v_i$ and $h_j$ are the states of the visible and hidden units.
# * $b_i$ and $c_j$ are the biases for the visible and hidden units, respectively.
# * $w_{ij}$ is the weight connecting visible unit $i$ to hidden unit $j$.
#
# The RBM learns by adjusting the weights and biases to assign low energy to the training data.
#
# ### Conditional Probabilities
#
# Due to the bipartite graph structure, the visible units are conditionally independent given the hidden units, and vice-versa. This allows us to calculate the probability of a unit's state based on the states of the units in the other layer using the logistic sigmoid function $\sigma(x) = \frac{1}{1 + e^{-x}}$.
#
# The conditional probability of hidden unit $h_j$ being active, given the visible vector $\mathbf{v}$, is:
#
# $$P(h_j=1|\mathbf{v}) = \sigma(c_j + \sum_{i=1}^{n} w_{ij} v_i)$$
#
# Similarly, the conditional probability of visible unit $v_i$ being active, given the hidden vector $\mathbf{h}$, is:
#
# $$P(v_i=1|\mathbf{h}) = \sigma(b_i + \sum_{j=1}^{m} w_{ij} h_j)$$
#
# ### RBM Implementation
#
# This is a complete implementation of the RBM class from the provided repository.

class RBM:
    """
    A Restricted Boltzmann Machine (RBM) implementation from scratch.

    An RBM is a two-layer neural network with a visible layer and a hidden layer.
    It learns a probability distribution over its inputs.
    """

    def __init__(self, visible_dim, hidden_dim, learning_rate=0.01, epochs=100, k=1):
        """
        Initializes the RBM with random weights and biases.

        Args:
            visible_dim (int): Number of units in the visible layer.
            hidden_dim (int): Number of units in the hidden layer.
            learning_rate (float): The learning rate for parameter updates.
            epochs (int): The number of training epochs.
            k (int): The number of Gibbs sampling steps for Contrastive Divergence.
        """
        self.visible_dim = visible_dim
        self.hidden_dim = hidden_dim
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.k = k

        # Initialize weights and biases. Weights are initialized with small
        # random values to break symmetry. Biases are initialized to zeros.
        self.W = np.random.randn(visible_dim, hidden_dim) * 0.01
        self.visible_bias = np.zeros(visible_dim)
        self.hidden_bias = np.zeros(hidden_dim)

    def _sigmoid(self, x):
        """
        The logistic sigmoid activation function.

        Args:
            x (np.ndarray): Input array.

        Returns:
            np.ndarray: The sigmoid of the input.
        """
        return 1.0 / (1.0 + np.exp(-x))

    def _propagate_up(self, v):
        """
        Propagates data from the visible layer to the hidden layer.
        This is also known as the 'positive phase' of the RBM.

        Args:
            v (np.ndarray): The visible layer vector.

        Returns:
            tuple: A tuple containing the hidden unit probabilities and
                   the sampled hidden unit states.
        """
        # Calculate hidden unit probabilities based on the conditional probability formula.
        hidden_prob = self._sigmoid(np.dot(v, self.W) + self.hidden_bias)
        # Sample the hidden unit states from a Bernoulli distribution.
        hidden_states = (hidden_prob > np.random.rand(self.hidden_dim)).astype(int)
        return hidden_prob, hidden_states

    def _propagate_down(self, h):
        """
        Propagates data from the hidden layer back to the visible layer.
        This is also known as the 'negative phase' of the RBM.

        Args:
            h (np.ndarray): The hidden layer vector.

        Returns:
            tuple: A tuple containing the visible unit probabilities and
                   the sampled visible unit states.
        """
        # Calculate visible unit probabilities based on the conditional probability formula.
        visible_prob = self._sigmoid(np.dot(h, self.W.T) + self.visible_bias)
        # Sample the visible unit states from a Bernoulli distribution.
        visible_states = (visible_prob > np.random.rand(self.visible_dim)).astype(int)
        return visible_prob, visible_states

    def _free_energy(self, v):
        """
        Calculates the free energy of the visible layer.
        The RBM is trained to minimize this value.
        """
        return -np.sum(np.dot(v, self.visible_bias)) - np.sum(np.log(1 + np.exp(np.dot(v, self.W) + self.hidden_bias)))

### CELL ###
# ## 3. The Training Algorithm: Contrastive Divergence
#
# Training an RBM means maximizing the likelihood of the training data. This is computationally intractable because it requires summing over all possible configurations of visible and hidden units. To solve this, we use an approximation called **Contrastive Divergence (CD-k)**.
#
# The core idea of CD-k is to perform a short, approximate Gibbs sampling process instead of a full, converged one. This is sufficient to get a good gradient estimate for training. The process has two phases:
#
# 1.  **Positive Phase:** Start with a training data vector $\mathbf{v}^{(0)}$ and propagate it up to the hidden layer to get probabilities $P(\mathbf{h}|\mathbf{v}^{(0)})$ and states $\mathbf{h}^{(0)}$. The goal of this phase is to increase the probability of this real data.
# 2.  **Negative Phase:** Propagate the sampled hidden states $\mathbf{h}^{(0)}$ back down to get a "reconstruction" of the visible data $\mathbf{v}^{(1)}$. Then, propagate this reconstructed visible data back up to the hidden layer to get states $\mathbf{h}^{(1)}$. This is a single Gibbs sampling step. The goal of this phase is to decrease the probability of this "fantasy" data.
#
# The learning is driven by the difference between the statistics of the positive and negative phases. The update rules for the weights and biases are:
#
# * **Weight Update:**
#     $$\Delta w_{ij} = \epsilon (\langle v_i h_j \rangle_{\text{data}} - \langle v_i h_j \rangle_{\text{reconstructed}})$$
# * **Visible Bias Update:**
#     $$\Delta b_i = \epsilon (\langle v_i \rangle_{\text{data}} - \langle v_i \rangle_{\text{reconstructed}})$$
# * **Hidden Bias Update:**
#     $$\Delta c_j = \epsilon (\langle h_j \rangle_{\text{data}} - \langle h_j \rangle_{\text{reconstructed}})$$
#
# Where $\epsilon$ is the learning rate, and $\langle \cdot \rangle$ denotes the expectation. We approximate these expectations with the values from our positive and negative phases. The `fit` method in our RBM class implements this logic.

def fit(self, X):
    """
    Trains the RBM using the Contrastive Divergence algorithm.

    Args:
        X (np.ndarray): The training data.
    """
    for epoch in range(self.epochs):
        # Mini-batch training for efficiency
        for i in range(0, len(X), 1):
            v0 = X[i:i+1]

            # Positive phase: Propagate real data up to hidden layer
            hidden_prob_0, hidden_states_0 = self._propagate_up(v0)

            # Negative phase (Gibbs sampling):
            # Propagate hidden states down to reconstruct visible states
            visible_prob_k, visible_states_k = self._propagate_down(hidden_states_0)
            
            # This is CD-1. For CD-k, we would run k steps of Gibbs sampling.
            # In this implementation, k is fixed at 1.

            # Update weights and biases based on the difference
            # between positive and negative phase statistics.
            # The matrix multiplication `v0.T @ hidden_prob_0` calculates
            # the correlation (v_i h_j) for the positive phase.
            positive_association = np.dot(v0.T, hidden_prob_0)
            
            # `visible_prob_k.T @ self._propagate_up(visible_states_k)[0]`
            # calculates the correlation for the negative phase.
            # This is the reconstructed data's "fantasy" correlation.
            negative_association = np.dot(visible_prob_k.T, self._propagate_up(visible_states_k)[0])

            # Apply the gradient updates
            self.W += self.learning_rate * (positive_association - negative_association)
            self.visible_bias += self.learning_rate * (np.sum(v0 - visible_prob_k, axis=0))
            self.hidden_bias += self.learning_rate * (np.sum(hidden_prob_0 - self._propagate_up(visible_states_k)[0], axis=0))

        # Print free energy for monitoring training progress.
        # This gives a sense of how well the model is learning the data distribution.
        if (epoch + 1) % 10 == 0:
            free_energy_value = np.mean([self._free_energy(sample) for sample in X])
            print(f"Epoch {epoch + 1}/{self.epochs}, Free Energy: {free_energy_value:.4f}")

### CELL ###
# ## 4. The Inference and Classification Pipeline
#
# Once the RBM is trained, it can be used for various tasks, including dimensionality reduction, feature extraction, and classification. For classification, we don't use the RBM directly. Instead, we use its hidden layer as a feature extractor. The trained RBM has learned meaningful representations of the data in its hidden units. We can then feed these representations into a separate, standard supervised classifier, such as Logistic Regression.
#
# This is a common "unsupervised pre-training" technique where the RBM acts as a feature-learning component before a final supervised classification layer.

def _transform(self, v):
    """
    Transforms visible data to hidden layer probabilities.
    This is used to get the feature representation for a classifier.

    Args:
        v (np.ndarray): The visible data to transform.

    Returns:
        np.ndarray: The hidden layer probabilities.
    """
    return self._sigmoid(np.dot(v, self.W) + self.hidden_bias)

# Add the _transform method to the RBM class
RBM._transform = _transform

### CELL ###
# ## 5. Putting It All Together: Main Execution
#
# Now we combine all the pieces to train the RBM and then classify the Iris data.

# --- Main Execution ---

# Define hyperparameters
VISIBLE_DIM = X_train.shape[1]
HIDDEN_DIM = 20  # A typical choice, can be tuned
LEARNING_RATE = 0.1
EPOCHS = 100

print("Initializing and training the RBM...")
rbm = RBM(
    visible_dim=VISIBLE_DIM,
    hidden_dim=HIDDEN_DIM,
    learning_rate=LEARNING_RATE,
    epochs=EPOCHS
)

# Train the RBM on the training data.
# The RBM learns features from the scaled numerical data.
rbm.fit(X_train)

# --- Classification with Logistic Regression ---

# Use the trained RBM to transform the training and test sets
# into a new, higher-level feature space.
X_train_rbm = rbm._transform(X_train)
X_test_rbm = rbm._transform(X_test)

# Now, we use a simple Logistic Regression model for classification
print("\nTraining a Logistic Regression model on RBM features...")
# `multi_class='multinomial'` and `solver='lbfgs'` are suitable for multi-class classification
classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the classifier on the new RBM features and the original labels.
classifier.fit(X_train_rbm, y_train)

# Make predictions on the test set.
y_pred = classifier.predict(X_test_rbm)

# Evaluate the final classification accuracy.
accuracy = accuracy_score(y_test, y_pred)

print(f"\nFinal Classification Accuracy: {accuracy:.2f}")
